# API requests for weather information

## 1. Importing libraries

In [ ]:
import pandas as pd
import requests
import re
import json
from datetime import datetime, timedelta, date
import sqlalchemy

## 2. Defining the flights funciton

In [ ]:
def tomorrows_flight_arrivals(icao_list):

    today = datetime.now().date()
    tomorrow = today + timedelta(days=1)

    list_for_arrivals_df = []

    for icao in icao_list:
        times = [["00:00","11:59"],["12:00","23:59"]]

        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {"direction":"Arrival","withCancelled":"false"}

            headers = {
                "X-RapidAPI-Key": "0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }

            response = requests.request("GET", url, headers=headers, params=querystring)
            flights_resp = response.json()

            arrivals_df = pd.json_normalize(flights_resp["arrivals"])[["number", "airline.name", "movement.scheduledTime.local", "movement.terminal", "movement.airport.name", "movement.airport.icao"]]
            arrivals_df = arrivals_df.rename(columns={"number": "flight_number", "airline.name": "airline", "movement.scheduledTime.local": "arrival_time", "movement.terminal": "arrival_terminal", "movement.airport.name": "departure_city", "movement.airport.icao": "departure_airport_icao"})
            arrivals_df['arrival_airport_icao'] = icao
            arrivals_df['data_retrieved_on'] = datetime.now().date()
            arrivals_df = arrivals_df[["flight_number", "departure_airport_icao", "arrival_airport_icao", "arrival_time"]]
            arrivals_df["arrival_time"] = arrivals_df["arrival_time"].str[:-6]
            list_for_arrivals_df.append(arrivals_df)

    return pd.concat(list_for_arrivals_df, ignore_index=True)

In [ ]:
icao_list = ["EDDB", "EDDF", "EDDK", "EDDM", "EDDH"]
airports_sql_df = tomorrows_flight_arrivals(icao_list).loc[:, ['arrival_airport_icao']]
airports_sql_df = airports_sql_df['arrival_airport_icao'].unique()
unique_airports_sql_df = pd.DataFrame({'arrival_airport_icao': airports_sql_df})

In [ ]:
unique_airports_sql_df

## 3. Pushing data to MySQL

In [ ]:
schema="gans" 
host="wbs-project3-db.cunuvsto5hvy.us-east-1.rds.amazonaws.com"
user="admin"
password = "***"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
unique_airports_sql_df.to_sql('airports',
                       if_exists='append',
                       con=con,
                       index=False)